In [1]:
import sys; sys.path.append('../../')

In [3]:
from jpdr.datasets import GroZi3k
import jpdr.transforms as T
from data import get_dataloaders

In [4]:
img_size = 800

dl_train_det, dl_train_recog, dl_val = get_dataloaders(
    'tonioni', img_size, normalize=False,
    shuffle_train=False, num_workers=8,
    train_crop_how="center"
)

In [5]:
from PIL import ImageDraw
from random import randint

def rand_chan():
    return randint(0, 255)

def rand_rgb():
    return (rand_chan(), rand_chan(), rand_chan())

def draw_boxes(im, boxes, labels=None):
    draw = ImageDraw.Draw(im)
    if labels is None:
        labels = [""]*len(boxes)

    for box, label in zip(boxes, labels):
        draw.rectangle(tuple(box), width=3, outline=rand_rgb())
        draw.text((box[0], box[1]), label, fill="white")

    return im

def add_label(im, label, xy=(10, 10)):
    draw = ImageDraw.Draw(im)
    draw.text(xy, label, fill="white")
    return im

In [6]:
from ipywidgets import interact, BoundedIntText
from torchvision.transforms import ToPILImage
from jpdr.utils.image_tools import show_ims

from training_steps_task_specific import create_recog_batch


def build_get_crop_boxes(dl, callback):
    def get_crop_boxes(
        batch=0,
    ):
        for i, (x, targets) in enumerate(dl):
            if i == batch:
                break

        crops, crop_tgts, crop_boxes = create_recog_batch(
            x, targets,
            return_crop_boxes=True
        )
        callback(x, targets, crops, crop_tgts, crop_boxes)

    return get_crop_boxes


def interact_crop_boxes(dl, callback):
    interact(
        build_get_crop_boxes(dl, callback),
        batch=BoundedIntText(
            value=0,
            min=0,
            max=len(dl),
        ),
    )


def show_crop_boxes(x, targets, crops, crop_tgts, crop_boxes):
    to_pil = ToPILImage()

    ncols = 2
    pil_ims = [
        draw_boxes(to_pil(t), boxes)
        for t, boxes in zip(x, crop_boxes)
    ]
    show_ims(pil_ims, figsize=(40, 40*len(pil_ims)/ncols), columns=ncols)


def show_crops_with_targets(x, targets, crops, tgt_ids, crop_boxes):
    to_pil = ToPILImage()
    pil_crops = [to_pil(crop) for crop in crops]

    crop_h, crop_w = crops.shape[-2:]
    ncols = 5
    scale = 0.1

    show_ims([
        add_label(to_pil(t), str(int(label)))
        for t, label in zip(crops, tgt_ids)
    ], figsize=(crop_w*scale, (len(pil_crops)*crop_h/ncols)*scale/4), columns=ncols)

In [7]:
batch = 0
dl = dl_train_recog

for i, (x, targets) in enumerate(dl):
    if i == batch:
        break

crops, crop_tgts, crop_boxes = create_recog_batch(
    x, targets,
    return_crop_boxes=True
)

In [8]:
interact_crop_boxes(dl_train_recog, show_crop_boxes)

interactive(children=(BoundedIntText(value=0, description='batch', max=49), Output()), _dom_classes=('widget-i…

In [9]:
interact_crop_boxes(dl_train_recog, show_crops_with_targets)

interactive(children=(BoundedIntText(value=0, description='batch', max=49), Output()), _dom_classes=('widget-i…